<a href="https://colab.research.google.com/github/hayannn/MeMI_ALFFEL_DATATHON/blob/main/%5Bkeyword1%EC%B0%A8%ED%95%84%ED%84%B0%EB%A7%81%5D_MeMI_ICU_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터셋 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
import os
import pandas as pd

# CSV 파일들이 위치한 디렉토리 경로
dir_path = '/content/drive/MyDrive/DataSet/Aiffel/Mimic/'

# 불러올 파일 리스트
target_files = ['ICUSTAYS.csv', 'PRESCRIPTIONS.csv', 'LABEVENTS.csv', 'ADMISSIONS.csv']

# 디렉토리 내 모든 파일 목록을 가져옴
files = os.listdir(dir_path)

# 지정된 파일만 불러오기
for file in files:
    file_path = os.path.join(dir_path, file)

    # 파일 이름이 target_files 목록에 있는지 확인
    if file in target_files and file.endswith('.csv'):
        try:
            # 파일을 읽어서 DataFrame으로 저장
            df = pd.read_csv(file_path, low_memory=False, on_bad_lines='skip')

            # 열 이름을 소문자로 변환
            df.columns = [col.lower() for col in df.columns]

            # 'row_id' 컬럼이 있으면 삭제
            if 'row_id' in df.columns:
                df = df.drop(columns=['row_id'])

            # DataFrame 이름을 파일 이름에 맞게 동적으로 할당
            dataframe_name = file.split('.')[0].lower()
            globals()[dataframe_name] = df

            print(f"Loaded DataFrame: {dataframe_name}, Shape: {df.shape}")

        except Exception as e:
            print(f"Error processing {file}: {e}")
    else:
        print(f"Skipping file: {file}")

Skipping file: CALLOUT.csv
Loaded DataFrame: admissions, Shape: (58976, 18)
Skipping file: CAREGIVERS.csv
Skipping file: D_CPT.csv
Skipping file: D_ICD_DIAGNOSES.csv
Skipping file: D_ICD_PROCEDURES.csv
Skipping file: D_ITEMS.csv
Skipping file: D_LABITEMS.csv
Skipping file: CPTEVENTS.csv
Skipping file: DIAGNOSES_ICD.csv
Skipping file: DRGCODES.csv
Loaded DataFrame: icustays, Shape: (61532, 11)
Skipping file: PATIENTS.csv
Skipping file: MICROBIOLOGYEVENTS.csv
Skipping file: PROCEDURES_ICD.csv
Skipping file: SERVICES.csv
Skipping file: TRANSFERS.csv
Skipping file: PROCEDUREEVENTS_MV.csv
Loaded DataFrame: prescriptions, Shape: (4156450, 18)
Skipping file: INPUTEVENTS_CV.csv
Skipping file: drug-ndc-0001-of-0001.json
Loaded DataFrame: labevents, Shape: (27854055, 8)


In [ ]:
# icustays = pd.read_csv(os.path.join(dir_path, 'ICUSTAYS.csv'), low_memory=False)
icustays.head()

,subject_id,hadm_id,icustay_id,los,hospital_expire_flag
0,268,110404,280836,3.2490,1
1,269,106296,206613,3.2788,0
2,270,188028,220345,2.8939,0
3,271,173727,249196,2.0600,0
4,272,164716,210407,1.6202,0


## Analyzing antibiotic usage in ICU patients and predicting treatment success

Step 1: Data Loading and Preprocessing

In [ ]:
# Merge ICUSTAYS with ADMISSIONS to include 'hospital_expire_flag'
icustays = icustays.merge(admissions[['subject_id', 'hadm_id', 'hospital_expire_flag']],
                          on=['subject_id', 'hadm_id'], how='left')

icustays.head()

,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los,hospital_expire_flag
0,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490,1
1,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788,0
2,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939,0
3,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600,0
4,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202,0


In [ ]:
# Filter ICU stays data
icustays = icustays[['subject_id', 'hadm_id', 'icustay_id', 'los', 'hospital_expire_flag']]

Step 2: Filter Antibiotics Data
- Extract antibiotic prescriptions from the PRESCRIPTIONS table.

In [ ]:
def data_rename(df, renames):
    renames_df = df[df['drug'].str.contains(renames, case=False, na=False)]
    unique_list = renames_df['drug'].unique()
    df['drug'] = df['drug'].replace(unique_list, renames)

    return df


In [ ]:
# # v1
# # Filter prescriptions for antibiotics
# antibiotics_keywords = ['cef', 'penicillin', 'amoxicillin', 'tetracycline', 'vancomycin', 'meropenem']
# antibiotics = prescriptions[prescriptions['drug'].str.contains('|'.join(antibiotics_keywords), case=False, na=False)]

# # Keep relevant columns
# antibiotics = antibiotics[['subject_id', 'hadm_id', 'startdate', 'enddate', 'drug', 'dose_val_rx']]

In [ ]:
# v2
# Filter prescriptions for antibiotics
antibiotics_keywords = ['GENTAMICIN', 'OXACILLIN', 'ERYTHROMYCIN', 'PENICILLIN',
       'LEVOFLOXACIN', 'NITROFURANTOIN', 'PIPERACILLIN/TAZO', 'MEROPENEM',
       'CEFTAZIDIME', 'CEFAZOLIN', 'CEFEPIME', 'TRIMETHOPRIM/SULFA',
       'TOBRAMYCIN', 'IMIPENEM', 'CEFTRIAXONE', 'CIPROFLOXACIN',
       'VANCOMYCIN', 'CLINDAMYCIN', 'TETRACYCLINE', 'RIFAMPIN',
       'CHLORAMPHENICOL', 'AMPICILLIN', 'LINEZOLID', 'PIPERACILLIN',
       'AMPICILLIN/SULBACTAM', 'CEFUROXIME', 'PENICILLIN G', 'DAPTOMYCIN',
       'AMIKACIN', 'CEFPODOXIME'
]


antibiotics = prescriptions[prescriptions['drug'].str.contains('|'.join(antibiotics_keywords), case=False, na=False)]

for name in antibiotics:
    antibiotics = data_rename(antibiotics, name)

# Keep relevant columns
antibiotics = antibiotics[['subject_id', 'hadm_id', 'startdate', 'enddate', 'drug', 'dose_val_rx']]

<ipython-input-19-ba4dfe59c094>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['drug'] = df['drug'].replace(unique_list, renames)


Step 3: Filter Infection-Related Lab Tests
- Extract relevant infection markers from the LABEVENTS table.

In [ ]:
# Filter LABEVENTS for relevant markers (e.g., WBC, CRP, Procalcitonin)
infection_markers = [51300, 51301, 51200, 51000]  # Replace with actual ITEMIDs from LABEVENTS
infection_tests = labevents[labevents['itemid'].isin(infection_markers)]

# Keep relevant columns
infection_tests = infection_tests[['subject_id', 'hadm_id', 'itemid', 'valuenum', 'charttime']]
infection_tests.head()

,subject_id,hadm_id,itemid,valuenum,charttime
52,3,NaN,51200,1.8,2101-10-14 03:00:00
69,3,NaN,51301,9.9,2101-10-14 03:00:00
101,3,NaN,51301,9.7,2101-10-15 03:30:00
136,3,NaN,51301,10.5,2101-10-16 04:00:00
190,3,145834.0,51200,2.8,2101-10-22 04:00:00


Step 4: Merge Data
- Combine ICU stay information, antibiotic usage, and lab results into a single dataset.

In [ ]:
# Merge ICU stays with antibiotics
data = icustays.merge(antibiotics, on=['subject_id', 'hadm_id'], how='inner')

# Merge with lab test results
data = data.merge(infection_tests, on=['subject_id', 'hadm_id'], how='left')

# Drop rows with missing survival information
data = data.dropna(subset=['hospital_expire_flag'])

# Ensure all date columns are datetime
data['startdate'] = pd.to_datetime(data['startdate'])
data['enddate'] = pd.to_datetime(data['enddate'])
data['charttime'] = pd.to_datetime(data['charttime'])
data.head()

,subject_id,hadm_id,icustay_id,los,hospital_expire_flag,startdate,enddate,drug,dose_val_rx,itemid,valuenum,charttime
0,268,110404,280836,3.249,1,2198-02-16,2198-02-18,Levofloxacin,250,51301.0,9.8,2198-02-16 20:50:00
1,268,110404,280836,3.249,1,2198-02-16,2198-02-18,Levofloxacin,250,51200.0,0.0,2198-02-17 02:57:00
2,268,110404,280836,3.249,1,2198-02-16,2198-02-18,Levofloxacin,250,51301.0,14.1,2198-02-17 02:57:00
3,268,110404,280836,3.249,1,2198-02-16,2198-02-18,Levofloxacin,250,51200.0,1.0,2198-02-11 10:40:00
4,268,110404,280836,3.249,1,2198-02-16,2198-02-18,Levofloxacin,250,51301.0,10.3,2198-02-11 10:40:00


Step 5: Feature Engineering

Prepare features for modeling, including:
- Aggregating lab test results.
- Creating antibiotic duration.

In [ ]:
# Calculate duration of antibiotic therapy
data['antibiotic_duration'] = (data['enddate'] - data['startdate']).dt.days

# Aggregate lab test results (mean and max values for each lab test per patient)
lab_features = data.groupby(['subject_id', 'hadm_id', 'itemid'])['valuenum'].agg(['mean', 'max']).unstack(fill_value=0)
lab_features.columns = ['_'.join(map(str, col)) for col in lab_features.columns]

# Merge lab features back into the dataset
data = data.groupby(['subject_id', 'hadm_id']).first().reset_index()
data = data.merge(lab_features, on=['subject_id', 'hadm_id'], how='left')

Step 6: Modeling

In [ ]:
# Prepare features and labels
X = data.drop(columns=['hospital_expire_flag', 'subject_id', 'hadm_id', 'icustay_id', 'drug'])
y = data['hospital_expire_flag']

- Random Forest

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_pred_rf_proba = rf_model.predict_proba(X_test)[:, 1]

print("Classification Report:\n", classification_report(y_test, y_pred_rf))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_rf_proba))

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.99      0.94      9784
           1       0.61      0.07      0.13      1375

    accuracy                           0.88     11159
   macro avg       0.75      0.53      0.53     11159
weighted avg       0.85      0.88      0.84     11159

ROC-AUC Score: 0.7436463242399466


In [ ]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)
y_pred_xgb_proba = xgb_model.predict_proba(X_test)[:, 1]

print("Classification Report:\n", classification_report(y_test, y_pred_xgb))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_xgb_proba))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:21:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      9784
           1       0.50      0.13      0.20      1375

    accuracy                           0.88     11159
   macro avg       0.69      0.56      0.57     11159
weighted avg       0.84      0.88      0.84     11159

ROC-AUC Score: 0.7470484650263881


- Random Forest Hyperparameter Tuning

In [ ]:
# Define the model
rf_model = RandomForestClassifier(random_state=42)

# Define hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Grid Search for hyperparameter tuning
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                              cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit model
grid_search_rf.fit(X_train, y_train)

# Get the best parameters
print("Best parameters:", grid_search_rf.best_params_)

# Use the best model
best_rf_model = grid_search_rf.best_estimator_

# Evaluate on test set
y_pred_rf = best_rf_model.predict(X_test)
y_pred_rf_proba = best_rf_model.predict_proba(X_test)[:, 1]

from sklearn.metrics import classification_report, roc_auc_score
print("Classification Report:\n", classification_report(y_test, y_pred_rf))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_rf_proba))

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best parameters: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.99      0.94      9784
           1       0.62      0.07      0.13      1375

    accuracy                           0.88     11159
   macro avg       0.75      0.53      0.53     11159
weighted avg       0.85      0.88      0.84     11159

ROC-AUC Score: 0.7510563814762506


# 하이퍼파라미터 튜닝
## GridSearch

- XGBoost Hyperparameter Tuning

In [ ]:
# Define the model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Define hyperparameters to tune
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Grid Search for hyperparameter tuning
grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb,
                               cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit model
grid_search_xgb.fit(X_train, y_train)

# Get the best parameters
print("Best parameters:", grid_search_xgb.best_params_)

# Use the best model
best_xgb_model = grid_search_xgb.best_estimator_

# Evaluate on test set
y_pred_xgb = best_xgb_model.predict(X_test)
y_pred_xgb_proba = best_xgb_model.predict_proba(X_test)[:, 1]

print("Classification Report:\n", classification_report(y_test, y_pred_xgb))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_xgb_proba))


Fitting 5 folds for each of 243 candidates, totalling 1215 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:24:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 300, 'subsample': 0.7}
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94      9784
           1       0.63      0.06      0.10      1375

    accuracy                           0.88     11159
   macro avg       0.76      0.53      0.52     11159
weighted avg       0.85      0.88      0.83     11159

ROC-AUC Score: 0.7640216680294359
